CNN으로 MNIST 분류 실습을 하자.  

### 1. 모델 이해하기

우리가 만들 모델의 아키텍처를 이해해보자.  
합성곱 신경망은 출처에 따라 합성곱 층을 부르는 단위가 조금 다르다.  

#### 1) 첫 번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고,  
맥스 풀링(nn.MaxPoold2d)는 풀링 층으로 별도로 명명한다.  

#### 2) 두 번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스 풀링(nn.MaxPoold2d)를  
하나의 합성곱 층으로 본다.  

본 챕터에는 편의상 맥스풀링도 포함하는 것으로 한다.

### 2. 모델 구현하기

#### 1) 필요한 도구 임포트, 입력의 정의

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 * 채널 * 높이(height) * 너비(width)의 크기의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)
print(f'텐서의 크기 : {inputs.shape}')

텐서의 크기 : torch.Size([1, 1, 28, 28])


#### 2) Convolution layer(합성곱층)과 Pooling

In [4]:
# 1채널, 32채널 출력, 커널 사이즈 3, 패딩 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# 32채널 입력, 64채널 출력, 커널 사이즈 3, 패딩 1
conv2 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


맥스풀링을 구현하자.  
정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정된다.

In [6]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


#### 3) 구현체를 연결하여 모델 만들기

이제 이들을 연결시켜서 모델을 완성시켜보자.  
입력을 첫 번째 합성곱층을 통과시키자.  

In [7]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28 X 28 텐서가 되었다.  
32채널은 conv1의 out_channel이 32로 설정했기 때문.  
크기가 28인 이유는 feature map 크기 계산 공식에 따라서 나옴.  

이제 맥스풀링에 통과시키고 텐서의 크기를 보자.

In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


stride가 2였기 때문에 28x28이 14x14가 됨.

이제 다시 두 번째 합성곱층에 통과시켜보자.

In [9]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


64가 나온 이유는 conv2의 out_chaannel을 64를 지정했기 때문이다.  
14 x 14가 된 이유는 마찬가지로 feature map 공식.

이제 다시 맥스풀링에 통과시켜 보자.  
역시 stride 2이기 때문에 절반이 되었다.

In [10]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 이 텐서를 펼치는 작업을 하자.  
이에 앞서 텐서의 n번째 차원에 접근하게 해주는 .size(n)에 대해서 배우자.  
현재 out의 크기는 [1, 64, 7, 7]이다.  
out의 첫 번째 차원이 몇인지 출력해보자.  

In [13]:
out.size(0)

1

In [14]:
out.size(1)

64

In [15]:
out.size(2)

7

In [16]:
out.size(3)

7

이제 이를 가지고 .view()를 사용하여 텐서를 펼치는 작업을 해보자.  

In [17]:
out = out.view(out.size(0), -1) # 첫 번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라.
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있다.  
이제 이에 대해서 Fully-Connected layer를 통과시켜보자.  

출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환하자.

In [18]:
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


### 3. CNN으로 MNIST 분류하기

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001   
training_epochs = 15
batch_size = 100

데이터셋을 정의한다.

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform = transforms.ToTensor(),
                         download=True)

100.0%
100.0%
100.0%
100.0%


데이터로더를 사용하여 배치 크기를 지정한다.

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

클래스로 모델을 설계한다.

In [6]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫 번째 층
        # ImgIn shape = (?, 28, 28 , 1)
        #  Conv      -> (?, 28, 28, 32)
        #  Pool      -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 두 번째 층
        # ImgIn shape = (?, 14, 14, 32)
        #  Conv       ->(?, 14, 14, 64)
        #  Pool       ->(?, 7,  7 , 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride= 2)

        )
        # 전결합층 7X7X64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(x)
        # out.size(0)는 첫 번째 차원인 배치차원.
        # out.size(0)만 유지하고, 나머지는 알아서 -> 즉 펼치는 셈이 된다.
        out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten

        out = self.fc(out)
        return out

모델을 선언한다.

In [7]:
# CNN 모델 선언
model =  CNN().to(device)

비용함수와 옵티마이저를 정의한다.

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스함수 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치 수를 출력해보자.

In [10]:
total_batch = len(data_loader)
print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


총 배치의 수가 600개이다.  
배치 크기를 100으로 했으므로, 즉 훈련 데이터가 60000개라는 의미이다.  
이제 모델을 훈련시켜보자.(colab으로 대체)

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0 # 에포크 당 평균 비용을 저장하기 위한 변수 초기화
    for X,Y in data_loader: # 미니 배치 단위로 데이터를 꺼내옴. X는 입력데이터, Y는 레이블
        # 이미지 데이터는 이미 28 * 28 크기를 가지므로, 별도의 reshape 필요없음
        # 레이블 Y는 원-핫 인코딩이 아닌 정수형 클래스 레이블임
        X = X.to(device) # 입력 데이터를 연산이 수행될 장치로 이동
        Y = Y.to(device) # 레이블을 연산이 수행될 장치로 이동

        optimizer.zero_grad() # 옵티마이저의 기울기 초기화
        hypothesis = model(X) # 모델을 통해 예측값을 계산(순전파 연산)
        cost = criterion(hypothesis, Y) # 예측값과 실제값 Y간의 cost 계산
        cost.backward()       # 역전파 연산을 통해 기울기 계산
        optimizer.step()      # 옵티마이저를 통해 파라미터 업데이트

        avg_cost += cost / total_batch  # 현재 배치의 비용을 전체 배치 수로 나누어 누적
    # 에포크가 끝날 때마다 평균 비용 출력
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

이제 테스트를 해보자

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad() 사용
with torch.no_grad():
    # 테스트 데이터를 모델에 입력하기 위한 준비
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동

    Y_test = mnist_test.test_labels.to(device) # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

    # 모델 예측 수행
    prediction = model(X_test) # 테스트 데이터에 대해 모델이 예측한 결과값

    # 예측 결과와 실제 레이블 비교
    # torch.argmax -> dim=1, 열방향으로 prediction의 최댓값들을 출력
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    # 예측된 클래스와 실제 레이블이 일치하는지 확인

    # 정확도 계산
    accuracy = correct_prediction.float().mean() # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함
    print('Accuracy:', accuracy.item())  # 정확도를 출력